### Setup

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.enable_eager_execution()

tf.__version__

'1.13.1'

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17859251695834247186
]


In [3]:
%run -i '../scripts/tf_test.py'

In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

## Start program

### Select a model

In [5]:
'''
Model 
'''

in_shape = (224,224,3)
img_size = [in_shape[0],in_shape[1]]

conv_base = tf.keras.applications.VGG16(weights='imagenet',
                                        input_shape=in_shape,
                                        include_top=False)

model = tf.keras.models.Sequential([
    conv_base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
#freezing the pretrained part of the model so it won't get updated as we train our model
print('This is the number of trainable weights before freezing the conv base: ', len(model.trainable_weights))
conv_base.trainable=False
print('This is the number of trainable weights after freezing the conv base: ', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base:  30
This is the number of trainable weights after freezing the conv base:  4


### Load Data

In [7]:
'''
Set Params

sample_size: 0-1
'''

sample_size = 1.0

batch_size = 32

In [8]:
'''
Load CSVs
'''

df = GetCSVs(sample_size)
print(df['Catenary'].value_counts())

img_paths,labels = GetPaths(df)

[Errno 2] File b'../data/output_images/China/CHN.csv' does not exist: b'../data/output_images/China/CHN.csv'
1    641
0    641
Name: Catenary, dtype: int64
1282


In [9]:
'''
Split, shuffle, and batch images into train/test datasets
'''

train_image_label_ds, val_image_label_ds, test_image_label_ds = SplitDataSet(img_paths, labels)

train_ds = ShuffleBatch(train_image_label_ds,len(img_paths),BATCH_SIZE = batch_size)
val_ds = ShuffleBatch(val_image_label_ds,len(img_paths),BATCH_SIZE = batch_size)
test_ds = ShuffleBatch(test_image_label_ds,len(img_paths),BATCH_SIZE = batch_size)

Number of images in train:  922
Distribution for train set:  (array([0, 1]), array([460, 462]))


Number of images in validation:  231
Distribution for validation set:  (array([0, 1]), array([113, 118]))


Number of images in test:  129
Distribution for test set:  (array([0, 1]), array([72, 57]))




### Compile, Fit, and Evaluate

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=5e-7),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [ ]:
# steps_epoch = int(tf.ceil(len(img_paths)/batch_size).numpy())
steps_epoch = 10

# num_epochs = steps_epoch*2
num_epochs = 10


history = model.fit(
    train_ds,
    steps_per_epoch=steps_epoch,
    epochs=num_epochs,
    validation_data=val_ds,
    validation_steps=steps_epoch)

Epoch 1/10
Instructions for updating:
Use tf.cast instead.
 7/10 [====================>.........] - ETA: 38s - loss: 0.7740 - acc: 0.4777

In [ ]:
#checking the result of the model.

acc = history.history['acc']
print('Accuracy:',acc)
val_acc = history.history['val_acc']

loss = history.history['loss']
print('Loss:',loss)
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
#To fine tune the model, unfroze last block of the pretrained model 
# since last block contains more complicated data
#that does not apply to most of the images unlike lower level that has edges 
# or other simpler data that can be reused for other classifications
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    elif layer.name == 'block5_conv2':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
#train the model again with partially unfrozen model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-8),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])


steps_epoch = 10

num_epochs = 10

history = model.fit(
    train_ds,
    steps_per_epoch=steps_epoch,
    epochs=num_epochs,
    validation_data=val_ds,
    validation_steps=steps_epoch)

In [ ]:
#checking the result of the model.

acc = history.history['acc']
print('Accuracy:',acc)
val_acc = history.history['val_acc']

loss = history.history['loss']
print('Loss:',loss)
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(
    test_ds,
    steps=steps_epoch)

### Save model

In [ ]:
#saving model for future use
filename = '../data/results/model_VGG_finetuned.h5'
model.save(filename)
print('Saved: %s' % filename)